# Hands on with PyMC
Let's practice everything we have learned this morning. We will be using the `PyMC` library to do Bayesian 
inference and explore posterior distributions. 

We only workout the case of two variants A and B with **Value conversions**. For now, we don't use real data yet and simply assume synthetic trials and successes.

## Value Conversions



Now what if we wanted to compare A/B test variants in terms of how much revenue they generate, and/or estimate how much additional revenue a winning variant brings? We can’t use a Beta-Binomial model for this, as the possible values for each visitor are now in the range [0, Inf).

The revenue generated by an individual visitor is 

revenue = probability of paying at all * mean amount spent when paying.

- As before, Bernoulli conversions follow Beta-Binomial.
- mean purchase amount follow a Gamma prior (which is also a conjugate prior to the Gamma likelihood)

- conversion of an individual follows Bernoulli(theta)
- amount spent by an individual follows Exp(lambda)

From N flips, z are head, which lead to payment. payment ~ gamma(z,lambda). Payment of each person ~Exp(lambda)

our **prior** about conversion chance of a user:

$$\theta_A \sim \theta_B \sim \mathrm{Beta}(\alpha_1, \beta_1)$$

our **prior** about the amount spent by a user:

$$\lambda_A \sim \lambda_B \sim \mathrm{Gamma}(\alpha_2, \beta_2)$$


conversion chance **Likelihood**:

$$conv_A \sim \mathrm{Binomial}(N_A, \theta_A)$$ 
$$conv_B \sim \mathrm{Binomial}(N_B, \theta_B)$$

amount spent **Likelihood**:

$$rev_A \sim \mathrm{Gamma}(conv_A, \lambda_A)$$
$$rev_B \sim \mathrm{Gamma}(conv_B, \lambda_B)$$

Average spending by two types_

$$\mu_A =  \dfrac{\theta_A}{\lambda_A}$$
$$\mu_B =  \dfrac{\theta_B}{\lambda_B}$$

uplift:

$$\mathrm{reluplift}_B = \mu_B / \mu_A - 1$$

$\mu$ here represents the average revenue per visitor, including those who don't make a purchase. This is the best way to capture the overall revenue effect - some variants may increase the average sales value, but reduce the proportion of visitors that pay at all (e.g. if we promoted more expensive items on the landing page).

Below we put the model setup into code and perform prior predictive checks.


In [ ]:
# Load libraries
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import numpy as np
import pymc as pm

In [ ]:
# Set random seed
rng = np.random.default_rng(1)
# Set plotting style
plotting_defaults = dict(bins=50,kind="hist",textsize=8,)

## Your First Try

In [ ]:
# as before we have variants A and B
variants  = ['A', 'B']

In [ ]:
# let's define the revenue data
# each variant has 1000 visitors 
visitors      = [1000, 1000]

# 200 of which leads to purchase
purchased     = [200, 200]

# and the total revenue is 1000
total_revenue = [200, 200]

In [ ]:
# let's define parameters for the mean purchase prior
purchase_alpha, purchase_beta = [9000, 900]

In [ ]:
# let's define parameters for a weak prior for the conversion rates
weak_alpha, weak_beta = [5000, 5000]

In [ ]:
with pm.Model() as example_model:

    # Priors for unknown model parameters
    theta = pm.Beta("theta", 
                    alpha = weak_alpha, 
                    beta  = weak_beta, 
                    shape = 2)
    
    # Priors for unknown model parameters
    lamda = pm.Gamma( "lamda", 
                    alpha = purchase_alpha,
                    beta  = purchase_beta,
                    shape = 2)

    # Likelihood (sampling distribution) of observations
    converted = pm.Binomial("converted", 
                            n        = visitors,      # total visitors
                            observed = purchased,     # total visitors converted
                            p        = theta,         # chance they convert
                            shape    = 2)  

    # Likelihood (sampling distribution) of observations
    revenue = pm.Gamma("revenue", 
                        alpha    = purchased,            # total visitors converted
                        observed = total_revenue, 
                        beta     = lamda, 
                        shape    = 2)        
    
    revenue_per_visitor = pm.Deterministic("revenue_per_visitor", 
                                           theta / lamda)


    theta_reluplift = []
    lam_reluplift   = []
    reluplift       = []

    for i in range(num_variants):
        others_theta = [theta[j] for j in range(num_variants) if j != i]
        others_lam   = [1 / lamd[j] for j in range(num_variants) if j != i]
        others_rpv   = [revenue_per_visitor[j] for j in range(num_variants) if j != i]
        
        if len(others_rpv) > 1:
            comparison_theta = pm.math.maximum(*others_theta)
            comparison_lam = pm.math.maximum(*others_lam)
            comparison_rpv = pm.math.maximum(*others_rpv)
        else:
            comparison_theta = others_theta[0]
            comparison_lam = others_lam[0]
            comparison_rpv = others_rpv[0]
            
        theta_reluplift.append(pm.Deterministic(f"theta_reluplift_{i}", theta[i] / comparison_theta - 1))
        lam_reluplift.append(pm.Deterministic(f"lam_reluplift_{i}", (1 / lamd[i]) / comparison_lam - 1))
        reluplift.append(pm.Deterministic(f"reluplift_{i}", revenue_per_visitor[i] / comparison_rpv - 1))

    revenue_prior_predictive = pm.sample_prior_predictive(samples=10000, return_inferencedata=False)

Sampling: [converted, lamd, revenue, theta]
